

## **Buisness Problem**: Retail Analysis (Cross-sell & Up-sell)



# **Dataset** 
(Online Retail.xlsx)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
! git clone https://github.com/Swapnil-Damate/Upselling-and-cross-selling-.git
#written by Swapnil


Cloning into 'Upselling-and-cross-selling-'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
dataset = pd.read_excel('/content/Upselling-and-cross-selling-/Online Retail.xlsx')
df = dataset.copy()
#written By Swapnil

## **Exploratory Data Analysis (EDA)**

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
 df=df[['CustomerID','InvoiceNo','StockCode','Quantity','UnitPrice','Description','InvoiceDate','Country']]

In [ ]:
       #Creating Total amount variable and adding it to dataset
TotalAmount=df['Quantity']*df['UnitPrice']
df.insert(loc=6,column='TotalAmount',value=TotalAmount)
df=df[['CustomerID','InvoiceNo','StockCode','Quantity','UnitPrice','TotalAmount','Description','InvoiceDate','Country']]
df.head()

,CustomerID,InvoiceNo,StockCode,Quantity,UnitPrice,TotalAmount,Description,InvoiceDate,Country
0,17850.0,536365,85123A,6,2.55,15.30,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,United Kingdom
1,17850.0,536365,71053,6,3.39,20.34,WHITE METAL LANTERN,2010-12-01 08:26:00,United Kingdom
2,17850.0,536365,84406B,8,2.75,22.00,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,United Kingdom
3,17850.0,536365,84029G,6,3.39,20.34,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,United Kingdom
4,17850.0,536365,84029E,6,3.39,20.34,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,United Kingdom


In [ ]:
                                                  #Total amount of purchases for each customer
new_df=df.groupby(['CustomerID']).sum()
new_df.sort_values('TotalAmount',ascending=False,inplace=True)
final_df = new_df[(new_df['Quantity']>0) & (new_df['TotalAmount'] > 0)]
final_df.head(10)

,Quantity,UnitPrice,TotalAmount
CustomerID,,,
14646.0,196719,5400.21,279489.02
18102.0,64122,5159.73,256438.49
17450.0,69029,3320.09,187482.17
14911.0,77180,31060.66,132572.62
12415.0,77242,2499.82,123725.45
14156.0,57025,6544.29,113384.14
17511.0,63012,2607.08,88125.38
16684.0,49390,709.93,65892.08
13694.0,61803,1163.81,62653.10


In [ ]:
dff = df.copy()
dff = dff.drop('CustomerID',axis=1)

In [ ]:
import seaborn as sns
sns.heatmap(dff.corr(),annot=True,cmap='viridis')

In [ ]:
                                                             #Customer who paid most and least
print('Customer ID {} had paid maximum amount of {}'.format(int(final_df['TotalAmount'].argmax()),final_df['TotalAmount'].max()))
print('Customer ID {} had paid minimum amount of {}'.format(int(final_df['TotalAmount'].argmin()),final_df['TotalAmount'].min()))

In [ ]:
                                                            #Maximum and Minimum no. of quantity and it's corresponding customer ID 
print('The maximum number of products has been bought is {} by customer id {}'.format(final_df['Quantity'].max(),final_df['Quantity'].argmax()))
print('The minimum number of product has been bought is {} by customer id {}'.format(final_df['Quantity'].min(),final_df['Quantity'].argmin()))

In [ ]:
                                            #Total sales at each country and how many quantities sold at each country
country=df.groupby(['Country']).sum()
country.sort_values('TotalAmount',ascending=False,inplace=True)
country.drop('CustomerID',axis=1,inplace=True)
country.head()

# **Data Cleaning / Data Preprocessing**

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
#To remove extra spaces in Description if present
df['Description'] = df['Description'].str.strip()

In [ ]:
#Drop NaN Entries
df.dropna()
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df['StockCode'] = df['StockCode'].astype('str')
df.shape

(541909, 9)

In [ ]:
#Remove cancelled/Retured Transaction
df = df[~df['InvoiceNo'].str.contains('C')]

df = df[~df['InvoiceNo'].str.contains('A')]


# **Feature Engineering**

In [ ]:

df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate']

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 532618, dtype: datetime64[ns]

In [ ]:
df['year']=df['InvoiceDate'].apply(lambda date:date.year)
df['month']=df['InvoiceDate'].apply(lambda date:date.month)

# **Evaluate Algorithm**

In [ ]:
!pip install mlxtend==0.17.0
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

In [ ]:
basket_Germany = (df[df['Country'] == 'Germany']
             .groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
basket_Germany.head()

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket_Germany.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

## Apriori Algorithm

In [ ]:
frequent_items = apriori(basket_sets, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_items, metric='lift', min_threshold=0.7)
rules.head()

In [ ]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.80) ]

# FpGrowth Algorithm

In [ ]:
frequent_items2 = fpgrowth(basket_sets,
                          min_support = 0.02, use_colnames = True)
rules2 = association_rules(frequent_items2, metric="lift", min_threshold=1)
rules2.head()

In [ ]:
Rules = rules2[ (rules2['lift'] >= 6) &
       (rules2['confidence'] >= 0.80) ]
Rules.head()

# Result Visualization

In [ ]:
#result.to_excel('/content/drive/My Drive/Internship Model Result/Final_retail_result.xlsx')
final_Table = pd.read_excel('/content/Upselling-and-cross-selling-/Final_onlineretail_result.xlsx')
final_Table = final_Table.drop('Unnamed: 0', axis=1)
final_Table = final_Table.drop('Unnamed: 0.1', axis=1)
final_Table = final_Table.replace()
final_Table

##**MODEL**

# CROSS-SELL

1.

In [ ]:
basket_Germany=basket_Germany.reset_index()
basket_Germany.head()

In [ ]:
data_ib = basket_Germany.drop("InvoiceNo", axis=1)

In [ ]:
# Create a placeholder dataframe listing item vs. item
data_ibs = pd.DataFrame(index=data_ib.columns,
                        columns=data_ib.columns)
data_ibs.head()

In [ ]:
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,5))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:4] = data_ibs.iloc[0:,i].sort_values(ascending=True)[:4].index
data_neighbours.head()

In [ ]:
data_neighbours.shape

2.

In [ ]:
!pip install implicit
from scipy import sparse
import scipy
import random
import implicit

     |████████████████████████████████| 1.1MB 3.5MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420226 sha256=a4c3e3d50e67637325ea1682676c75632b310f877d8ae5f5748c2ab04089ec0c
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [ ]:
recData = df.groupby(['CustomerID','StockCode','Description'], as_index = False).agg({'InvoiceNo': 'nunique'})

recData.rename(columns = {'InvoiceNo':'score'}, inplace = True)
recData.head()

,CustomerID,StockCode,Description,score
0,12346.0,23166,MEDIUM CERAMIC TOP STORAGE JAR,1
1,12347.0,16008,SMALL FOLDING SCISSOR(POINTED EDGE),1
2,12347.0,17021,NAMASTE SWAGAT INCENSE,1
3,12347.0,20665,RED RETROSPOT PURSE,1
4,12347.0,20719,WOODLAND CHARLOTTE BAG,4


In [ ]:
pivotData = pd.pivot_table(recData, values = 'score', index = 'CustomerID', 
                            columns = 'Description', aggfunc = np.sum)
masked = np.ma.masked_invalid(pivotData)
mc = masked.compressed()
print(1 - mc.shape[0]/np.prod(pivotData.shape))

# The maximum recommended sparsity is 99.5%

0.9840132452842022


In [ ]:
recData['CustomerID'] = recData['CustomerID'].astype("category")
recData['StockCode'] = recData['StockCode'].astype("category")
recData['user'] = recData['CustomerID'].cat.codes
recData['item_code'] = recData['StockCode'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparseItemUser = scipy.sparse.csr_matrix((recData['score'].astype(float), (recData['item_code'], recData['user'])))
sparseUserItem = scipy.sparse.csr_matrix((recData['score'].astype(float), (recData['user'], recData['item_code'])))

# Initialize the als model and fit it using the sparse item-user matrix
model_crosssell = implicit.als.AlternatingLeastSquares(factors=32, regularization=0.1, iterations=80)

# Calculate the confidence by multiplying it by alpha value, based on the methodologies of 
# http://yifanhu.net/PUB/cf.pdf
alphaVal = 15
dataConf = (sparseItemUser * alphaVal).astype('double')

#Fit the model
model_crosssell.fit(dataConf)

In [ ]:
recData.dropna()

,CustomerID,StockCode,Description,score,user,item_code
0,12346.0,23166,MEDIUM CERAMIC TOP STORAGE JAR,1,0,1997
1,12347.0,16008,SMALL FOLDING SCISSOR(POINTED EDGE),1,1,25
2,12347.0,17021,NAMASTE SWAGAT INCENSE,1,1,87
3,12347.0,20665,RED RETROSPOT PURSE,1,1,130
4,12347.0,20719,WOODLAND CHARLOTTE BAG,4,1,167
...,...,...,...,...,...,...
268354,18287.0,84920,PINK FLOWER FABRIC PONY,1,4338,3077
268355,18287.0,85039A,SET/4 RED MINI ROSE CANDLE IN BOWL,2,4338,3179
268356,18287.0,85039B,S/4 IVORY MINI ROSE CANDLE IN BOWL,2,4338,3180
268357,18287.0,85040A,S/4 PINK FLOWER CANDLES IN BOWL,1,4338,3182


In [ ]:


def getItemDescription(itemcode):
    """
    returns a list of descriptions given a list of items
    
    items: list of item categorical code
    """
    descriptions = []
    for item_code in itemcode:
        descriptions.extend(recData[recData['item_code'] == item_code ]['Description'].astype('str').unique())
    return descriptions

def getSimilarItems(item_code):
    """
    returns a dictionary of item categorical code and score
    
    item: item categorical code
    """
    similar = model_crosssell.similar_items(item_code, 11)
    similar.pop(0)
    similarItems = {}
    for item_code in similar:
        similarItems[item_code[0]] = item_code[1]
    return similarItems

In [ ]:
print("Original item:", getItemDescription([975]))
print("recommended item:", getItemDescription(getSimilarItems(975).keys()))

Original item: ['6 RIBBONS EMPIRE']
recommended item: ['6 RIBBONS SHIMMERING PINKS', '6 RIBBONS RUSTIC CHARM', 'SCANDINAVIAN REDS RIBBONS', 'CHOCOLATE BOX RIBBONS', 'ROMANTIC PINKS RIBBONS', 'URBAN BLACK RIBBONS', 'TRADITIONAL CHRISTMAS RIBBONS', 'RIBBON REEL LACE DESIGN', '6 RIBBONS ELEGANT CHRISTMAS', 'MEASURING TAPE BABUSHKA BLUE']


In [ ]:
def reccomend_user(i):
  print("Original item:", getItemDescription([i]))
  print("recommended item for CROOSSELL:", getItemDescription(getSimilarItems(i).keys()))

In [ ]:
reccomend_user(975)

Original item: ['6 RIBBONS EMPIRE']
recommended item for CROOSSELL: ['6 RIBBONS SHIMMERING PINKS', '6 RIBBONS RUSTIC CHARM', 'SCANDINAVIAN REDS RIBBONS', 'CHOCOLATE BOX RIBBONS', 'ROMANTIC PINKS RIBBONS', 'URBAN BLACK RIBBONS', 'TRADITIONAL CHRISTMAS RIBBONS', 'RIBBON REEL LACE DESIGN', '6 RIBBONS ELEGANT CHRISTMAS', 'MEASURING TAPE BABUSHKA BLUE']


# UPSELL

In [ ]:
!pip install gensim
!pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=dataset.copy()

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.shape

(541909, 8)

In [ ]:
# Preprocessing
df['clean_products'] = df['Description'].str.replace("[^a-zA-Z#]", " ")
df['clean_products'] = df['clean_products'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_products'] = df['clean_products'].fillna('').apply(lambda x: x.lower())
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,clean_products
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,white hanging heart light holder
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,white metal lantern
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,cream cupid hearts coat hanger
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,knitted union flag hot water bottle
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,red woolly hottie white heart
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,pack spaceboy napkins
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,children apron dolly girl
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,childrens cutlery dolly girl
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,childrens cutlery circus parade


In [ ]:
stop_words = stopwords.words('english')

# tokenization
tokenized_doc = df['clean_products'].fillna('').apply(lambda x: x.split())

# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_products'] = detokenized_doc
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,clean_products
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,white hanging heart light holder
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,white metal lantern
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,cream cupid hearts coat hanger
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,knitted union flag hot water bottle
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,red woolly hottie white heart
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,pack spaceboy napkins
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,children apron dolly girl
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,childrens cutlery dolly girl
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,childrens cutlery circus parade


In [ ]:
df.isnull().sum()

InvoiceNo              0
StockCode              0
Description         1454
Quantity               0
InvoiceDate            0
UnitPrice              0
CustomerID        135080
Country                0
clean_products         0
dtype: int64

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

InvoiceNo         0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
clean_products    0
dtype: int64

In [ ]:
df['StockCode']= df['StockCode'].astype(str)

In [ ]:
customers = df["CustomerID"].unique().tolist()
len(customers)

4372

In [ ]:
select=df[['StockCode','CustomerID','clean_products','UnitPrice','Description']]
df=select.copy()
df

In [ ]:
#swapnil
recData = recData.dropna()
df['item_code'] = recData['item_code']
df['item_code'] = df['item_code']
pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
#swapnil
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,clean_products,item_code
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,"17,850",United Kingdom,white hanging heart light holder,"1,997"
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,"17,850",United Kingdom,white metal lantern,25
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,"17,850",United Kingdom,cream cupid hearts coat hanger,87
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,"17,850",United Kingdom,knitted union flag hot water bottle,130
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,"17,850",United Kingdom,red woolly hottie white heart,167


In [ ]:
#purchases= []

# populate the list with the product codes
#for i in tqdm(customers):
#    temp = df[df["CustomerID"] == i]["StockCode"].tolist()
#    purchases.append(temp)

In [ ]:
#swapnil
purchases= []

# populate the list with the product codes
for i in tqdm(customers):
    temp = df[df["CustomerID"] == i]["item_code"].tolist()
    purchases.append(temp)

100%|██████████| 4372/4372 [00:06<00:00, 721.44it/s]


In [ ]:
print(purchases[0:2])


[[1997.0, 25.0, 87.0, 130.0, 167.0, 206.0, 208.0, 274.0, 315.0, 1576.0, 1577.0, 1578.0, 1620.0, 1621.0, 1622.0, 1623.0, 1624.0, 1651.0, 1667.0, 1787.0, 1833.0, 1911.0, 1919.0, 1977.0, 1978.0, 1993.0, 2001.0, 2003.0, 2004.0, 2005.0, 2006.0, 2008.0, 2093.0, 2115.0, 2126.0, 2134.0, 2229.0, 2230.0, 2231.0, 2287.0, 2303.0, 2309.0, 2312.0, 2591.0, 2595.0, 2504.0, 138.0, 139.0, 140.0, 170.0, 178.0, 179.0, 341.0, 382.0, 427.0, 438.0, 439.0, 440.0, 441.0, 442.0, 443.0, 445.0, 531.0, 1482.0, 1483.0, 436.0, 437.0, 553.0, 602.0, 613.0, 614.0, 615.0, 634.0, 824.0, 825.0, 841.0, 926.0, 927.0, 928.0, 929.0, 930.0, 931.0, 932.0, 956.0, 615.0, 641.0, 784.0, 880.0, 930.0, 1350.0, 1482.0, 1483.0, 1484.0, 1564.0, 1666.0, 1802.0, 1954.0, 1999.0, 2070.0, 2082.0, 2084.0, 2089.0, 2098.0, 2099.0, 2100.0, 2107.0, 2108.0, 2109.0, 2110.0, 2110.0, 2115.0, 2137.0, 910.0, 1025.0, 1407.0, 1411.0, 1412.0, 2593.0, 2594.0, 2605.0, 2606.0, 2613.0, 3664.0, 12.0, 46.0, 138.0, 139.0, 140.0, 146.0, 161.0, 347.0, 348.0, 361.0

In [ ]:
model = Word2Vec(window = 10, sg = 1,negative = 10) # for negative sampling

model.build_vocab(purchases, progress_per=200)

model.train(purchases, total_examples = model.corpus_count, epochs=10)

(2002699, 4068290)

In [ ]:
model.init_sims(replace=True)
print(model)

Word2Vec(vocab=2995, size=100, alpha=0.025)


In [ ]:
# extract all vectors
X = model[model.wv.vocab]
X.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(2995, 100)

In [ ]:
#products = df[["StockCode", "Description","UnitPrice"]].copy()
#products

In [ ]:
#swapnil
products = df[["item_code", "Description","UnitPrice"]].copy()
products =products.dropna()
products

,item_code,Description,UnitPrice
0,1997.0,WHITE HANGING HEART T-LIGHT HOLDER,2.55
1,25.0,WHITE METAL LANTERN,3.39
2,87.0,CREAM CUPID HEARTS COAT HANGER,2.75
3,130.0,KNITTED UNION FLAG HOT WATER BOTTLE,3.39
4,167.0,RED WOOLLY HOTTIE WHITE HEART.,3.39
...,...,...,...
541904,nan,PACK OF 20 SPACEBOY NAPKINS,0.85
541905,nan,CHILDREN'S APRON DOLLY GIRL,2.10
541906,nan,CHILDRENS CUTLERY DOLLY GIRL,4.15
541907,nan,CHILDRENS CUTLERY CIRCUS PARADE,4.15


In [ ]:
#products.drop_duplicates(inplace=True, subset='StockCode', keep="last")
#products_dict = products.groupby('StockCode')["Description"].apply(list).to_dict()
#products_dict

In [ ]:
#swapnil
products.drop_duplicates(inplace=True, subset='item_code', keep="last")
products_dict = products.groupby('item_code')["Description"].apply(list).to_dict()
products_dict

{'0.0': ['CARD WEDDING DAY'],
 '1.0': ['SMALL DECO JEWELLERY STAND'],
 '10.0': ['SPOTTY BUNTING'],
 '100.0': ['CHEST OF DRAWERS GINGHAM HEART '],
 '1000.0': ['SET OF 60 PANTRY DESIGN CAKE CASES '],
 '1001.0': ['PACK OF 12 TRADITIONAL CRAYONS'],
 '1002.0': ['MINT KITCHEN SCALES'],
 '1003.0': ['CHARLOTTE BAG SUKI DESIGN'],
 '1004.0': ['36 FOIL HEART CAKE CASES'],
 '1005.0': ['SET 20 NAPKINS FAIRY CAKES DESIGN '],
 '1006.0': ['WOODLAND CHARLOTTE BAG'],
 '1007.0': ['CHILDRENS CUTLERY CIRCUS PARADE'],
 '1008.0': ['SET OF 3 REGENCY CAKE TINS'],
 '1009.0': ['MINT KITCHEN SCALES'],
 '101.0': ['LOVE BUILDING BLOCK WORD'],
 '1010.0': ['ZINC T-LIGHT HOLDER STARS SMALL'],
 '1011.0': ['RECYCLED ACAPULCO MAT TURQUOISE'],
 '1012.0': ['SET/5 RED RETROSPOT LID GLASS BOWLS'],
 '1013.0': ['WOODEN FRAME ANTIQUE WHITE '],
 '1014.0': ['GREEN BIRDHOUSE DECORATION'],
 '1015.0': ['RED GINGHAM ROSE JEWELLERY BOX'],
 '1016.0': ['PACK OF 12 RED RETROSPOT TISSUES '],
 '1017.0': ['GINGHAM HEART DECORATION'],
 '1018

In [ ]:
#def similar_products(no, n = 10):
 #   v=model[no]
    # extract most similar products for the input vector
#    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
#    new_ms = []
#    for j in ms:
#        k=products.loc[products['StockCode'] == j[0],'UnitPrice'].tolist()
#        pair = (j[0], products_dict[ j[0]][0], k[0], j[1])
 #       new_ms.append(pair)
    
 #   return pd.DataFrame(new_ms,columns=['StockCode','Product','Price','Weightage']).sort_values(by=['Price'],ascending=False).\
 #                   set_index('StockCode')

In [ ]:
#swapnil
def similar_products(no, n = 10):
    v=model[no]
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        k=products.loc[products['item_code'] == j[0],'UnitPrice'].tolist()
        pair = (j[0], products_dict[ j[0]][0], k[0], j[1])
        new_ms.append(pair)
    
    return pd.DataFrame(new_ms,columns=['item_code','Product','Price','Weightage']).sort_values(by=['Price'],ascending=False).\
                    set_index('item_code')

In [ ]:
#def product_detail(var): 
#    prod_name=products_dict[var]
 #   prod_value=products.loc[products['StockCode'] == var,'UnitPrice'].tolist()
 #   list_of_tuples = [(var,prod_name[0],prod_value[0])]
 #   return pd.DataFrame(list_of_tuples,columns=['StockCode','Description','Price']).set_index('StockCode') 

In [ ]:
#swapnil
def product_detail(var): 
    prod_name=products_dict[var]
    prod_value=products.loc[products['item_code'] == var,'UnitPrice'].tolist()
    list_of_tuples = [(var,prod_name[0],prod_value[0])]
    return pd.DataFrame(list_of_tuples,columns=['item_code','Description','Price']).set_index('item_code') 

In [ ]:
#product_detail('85123A') 

In [ ]:
#similar_products('85123A')

In [ ]:
#product_detail('16218')
#similar_products('16218')

In [ ]:
#product_detail('22464')
#similar_products('22464')

In [ ]:
#swapnil
#product_detail('167.0')
similar_products('379.0')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,Product,Price,Weightage
item_code,,,
318.0,RED CHARLIE+LOLA PERSONAL DOORSIGN,4,1
316.0,SMALL PARISIENNE HEART PHOTO FRAME,4,1
352.0,EDWARDIAN PARASOL PINK,4,1
408.0,PARTY BUNTING,3,1
407.0,JAM JAR WITH PINK LID,2,1
370.0,FAIRY TALE COTTAGE NIGHTLIGHT,2,1
383.0,PLASTERS IN TIN CIRCUS PARADE,2,1
349.0,GARDENERS KNEELING PAD CUP OF TEA,1,1
372.0,DINOSAUR KEYRINGS ASSORTED,1,1


## FINAL MODEL

In [146]:
def reccomend(i):
  print("Original item:", getItemDescription([i]))
  print("....................................")
  print("recommended item for CROOSSELL:", getItemDescription(getSimilarItems(i).keys()))
  print("....................................")
  print(".")
  print(".")
  print(".")
  print(".")
  print("recommended item for UP SELL:")
  a=float(i)
  a=str(a)
  b = a
  v=model[b]
  n=5
    # extract most similar products for the input vector
  ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
  new_ms = []
  for j in ms:
    k=products.loc[products['item_code'] == j[0],'UnitPrice'].tolist()
    pair = (j[0], products_dict[ j[0]][0], k[0], j[1])
    new_ms.append(pair)
    
  return pd.DataFrame(new_ms,columns=['item_code','Product','Price','Weightage']).sort_values(by=['Price'],ascending=False).\
                    set_index('item_code')
  

In [151]:
reccomend(194)

Original item: ['GARDEN PATH SKETCHBOOK']
....................................
recommended item for CROOSSELL: ['BLUE PAISLEY SKETCHBOOK', 'GREEN FERN SKETCHBOOK', 'ABSTRACT CIRCLES SKETCHBOOK', 'CHRYSANTHEMUM SKETCHBOOK', 'CHRYSANTHEMUM NOTEBOOK', 'BLUE PAISLEY POCKET BOOK', 'ABSTRACT CIRCLE JOURNAL', 'CHRYSANTHEMUM  JOURNAL', 'BLUE PAISLEY JOURNAL', 'GREEN FERN JOURNAL']
....................................
.
.
.
.
recommended item for UP SELL:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,Product,Price,Weightage
item_code,,,
196.0,YULETIDE IMAGES GIFT WRAP SET,10,1
197.0,TEA BAG PLATE RED RETROSPOT,8,1
199.0,PACK OF 12 TRADITIONAL CRAYONS,4,1
193.0,WRAP ENGLISH ROSE,2,1
192.0,MAGNETS PACK OF 4 SWALLOWS,0,1
